In [163]:
import requests
from io import BytesIO
from PIL import Image

import numpy as np
from matplotlib import pyplot as plt

import torch
import torchvision.transforms as transforms
from torchvision import models

import pandas as pd
import numpy as np
import time

In [165]:
df = pd.read_csv("../data/silver_20250322_Airlines_Reviews_Sentiment.csv")
df

,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended,sentiment_label,sentiment_score
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Solo Leisure,...,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes,positive,0.918486
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,NaN,E120,Solo Leisure,...,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no,negative,0.569241
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,NaN,Embraer E120,Solo Leisure,...,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no,neutral,0.542810
3,3,Adria Airways,1,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,NaN,NaN,Solo Leisure,...,1.0,1.0,NaN,1.0,NaN,NaN,1.0,no,negative,0.903330
4,4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,NaN,Couple Leisure,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no,negative,0.902472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24349,24349,ZIPAIR,1,"""customer service is terrible""",5th July 2022,False,Bangkok to Tokyo. I’ve flown many low cost ai...,NaN,NaN,Couple Leisure,...,2.0,1.0,NaN,1.0,1.0,1.0,1.0,no,negative,0.842372
24350,24350,ZIPAIR,1,"""Avoid at all costs""",1st June 2022,True,Avoid at all costs. I booked flights to go f...,NaN,NaN,Solo Leisure,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,no,negative,0.836562
24351,24351,ZIPAIR,3,"""Will not recommend to anyone""",31st May 2022,True,Flight was leaving at 23.15 and after an hou...,NaN,NaN,Business,...,2.0,4.0,2.0,1.0,NaN,NaN,2.0,no,negative,0.797939
24352,24352,ZIPAIR,6,"""It was immaculately clean""",23rd May 2022,True,Zipair is JAL’s budget airline. They don’t ha...,NaN,Dreamliner,Business,...,3.0,4.0,3.0,1.0,2.0,5.0,5.0,yes,neutral,0.654033


In [167]:
# Remove this when 2_DataPreprocessing step is added
df.rename(columns={"Unnamed: 0": "RowId"}, inplace=True)

In [169]:
df_images = df[df['Top Review Image Url'].notnull()].copy()
df_images

,RowId,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended,sentiment_label,sentiment_score
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Solo Leisure,...,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes,positive,0.918486
13,13,Adria Airways,2,"""Would not fly again""",30th June 2018,True,Ljubljana to Munich. Adria's checkin system ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Business,...,1.0,2.0,2.0,2.0,NaN,NaN,1.0,no,negative,0.889687
108,108,Aegean Airlines,5,“claims to be pet-friendly”,27th October 2024,True,While my journey with Aegean Airlines started...,https://www.airlinequality.com/wp-content/uplo...,A320,Business,...,2.0,4.0,4.0,1.0,NaN,1.0,1.0,no,negative,0.802881
109,109,Aegean Airlines,9,“Great value for money!”,13th October 2024,True,Upgraded to business with Plusgrade. Great va...,https://www.airlinequality.com/wp-content/uplo...,A321neo,Business,...,5.0,5.0,5.0,4.0,4.0,4.0,3.0,yes,positive,0.951761
110,110,Aegean Airlines,1,"""75 euro for 1 10 kg bag round trip""",4th October 2024,True,From the bag check showing up 45 min late (...,https://www.airlinequality.com/wp-content/uplo...,NaN,Couple Leisure,...,1.0,1.0,1.0,1.0,NaN,NaN,1.0,no,negative,0.750796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24298,24298,ZIPAIR,1,"""being periodically harassed and ridiculed""",19th September 2023,True,My review lays with being periodically haras...,https://www.airlinequality.com/wp-content/uplo...,NaN,Solo Leisure,...,1.0,1.0,3.0,1.0,1.0,1.0,1.0,no,neutral,0.654004
24304,24304,ZIPAIR,1,"""can highly recommend ZIPAIR""",23rd July 2023,True,A great experience! Everything was easy. Trav...,https://www.airlinequality.com/wp-content/uplo...,NaN,Family Leisure,...,5.0,5.0,NaN,5.0,5.0,5.0,5.0,yes,positive,0.972730
24312,24312,ZIPAIR,1,"""I heard back 3 months later""",18th May 2023,True,ZIPAIR completely broke my brand new luggage ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Couple Leisure,...,2.0,3.0,3.0,2.0,1.0,1.0,2.0,no,negative,0.926751
24320,24320,ZIPAIR,1,ZIPAIR customer review,21st April 2023,True,The seats are abysmal for a long-haul flight...,https://www.airlinequality.com/wp-content/uplo...,Boeing 787,Couple Leisure,...,1.0,2.0,1.0,1.0,NaN,1.0,1.0,no,negative,0.907984


#### Get imagenet_classes

In [172]:
imagenet_classes = (requests
           .get("https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json")
           .json())
imagenet_classes = np.array(imagenet_classes)
imagenet_classes

array(['tench', 'goldfish', 'great white shark', 'tiger shark',
       'hammerhead shark', 'electric ray', 'stingray', 'cock', 'hen',
       'ostrich', 'brambling', 'goldfinch', 'house finch', 'junco',
       'indigo bunting', 'American robin', 'bulbul', 'jay', 'magpie',
       'chickadee', 'American dipper', 'kite', 'bald eagle', 'vulture',
       'great grey owl', 'fire salamander', 'smooth newt', 'newt',
       'spotted salamander', 'axolotl', 'American bullfrog', 'tree frog',
       'tailed frog', 'loggerhead sea turtle', 'leatherback sea turtle',
       'mud turtle', 'terrapin', 'box turtle', 'banded gecko',
       'green iguana', 'Carolina anole',
       'desert grassland whiptail lizard', 'agama',
       'frilled-necked lizard', 'alligator lizard', 'Gila monster',
       'European green lizard', 'chameleon', 'Komodo dragon',
       'Nile crocodile', 'American alligator', 'triceratops',
       'worm snake', 'ring-necked snake', 'eastern hog-nosed snake',
       'smooth green snak

#### Classify images using RESNET50 top 1 class

In [237]:
def resnet50_classify_image(image_url, row_id=None):
    print(row_id, image_url)
    
    model = models.resnet50(weights="IMAGENET1K_V2")
    model.eval()  # Set to evaluation mode
    
    # Get an image from the internet
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    
    # Define transformation to match ResNet50's expected input
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    score = -1
    label = 'error'
    try:
        # Apply transformation
        x = transform(img).unsqueeze(0)  # Add batch dimension
        
        # Run the image through the model
        y_hat = model(x)
    
        # Extracts the predictions (logits) as a NumPy array, sorts them in descending order, 
        # and retrieves indices corresponding to the top predicted classes.
        y_hat = y_hat.detach().numpy().squeeze()
        I = y_hat.argsort()[::-1]
    
        #1-second delay so as not to spam/DOS the source webserver
        time.sleep(1)
    
        print(I[:5]) # Top 5 class indices
        print(y_hat[I[:5]]) # Probabilities/logits for top 5 classes
        print(imagenet_classes[I[:5]]) # Top 5 class names (from ImageNet)
    
        score = y_hat[I[:1]].item()
        label = imagenet_classes[I[:1]].item()
    except RuntimeError as e:
        print(str(e))
        
    return score, label

In [239]:
resnet50_classify_image('https://www.airlinequality.com/wp-content/uploads/2018/06/IMG_1571-1-500x500.jpg', 13)

13 https://www.airlinequality.com/wp-content/uploads/2018/06/IMG_1571-1-500x500.jpg
[636 748 785 464 885]
[6.6289973 4.485667  3.5766733 3.1832204 3.0856068]
['mail bag' 'purse' 'seat belt' 'buckle' 'velvet']


(6.628997325897217, 'mail bag')

In [196]:
''' Alternate
df_images[['img_resnet50_score', 'img_resnet50_label']] = df_images.apply(
    lambda row: pd.Series(resnet50_classify_image(row['Top Review Image Url'], row['RowId'])), axis=1
)
'''

0 https://www.airlinequality.com/wp-content/uploads/2019/11/20191102_123305-500x500.jpg
[895 908 404 403 657]
[5.190325  3.7582083 3.6045265 3.3716395 2.8360355]
['military aircraft' 'wing' 'airliner' 'aircraft carrier' 'missile']
13 https://www.airlinequality.com/wp-content/uploads/2018/06/IMG_1571-1-500x500.jpg
[636 748 785 464 885]
[6.6289973 4.485667  3.5766733 3.1832204 3.0856068]
['mail bag' 'purse' 'seat belt' 'buckle' 'velvet']
108 https://www.airlinequality.com/wp-content/uploads/2024/10/IMG_5873-500x500.jpeg
[609 545 791 790 581]
[5.0405984 4.7803736 4.7432065 4.4619155 4.0520296]
['jeep' 'electric fan' 'shopping cart' 'shopping basket' 'grille']
109 https://www.airlinequality.com/wp-content/uploads/2024/10/20241008_FRA-ATH-A3831_espresso-500x500.jpg
[967 968 504 910 785]
[7.355824  3.2939284 2.6654654 2.450601  2.326919 ]
['espresso' 'cup' 'coffee mug' 'wooden spoon' 'seat belt']
110 https://www.airlinequality.com/wp-content/uploads/2024/10/PXL_20241004_073123468-500x500.jpg

RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

In [200]:
# Define a function that processes each row
def classify_image(row_id, image_url):
    return resnet50_classify_image(image_url, row_id)

In [202]:
# Apply the function across the relevant columns
results = [
    classify_image(row_id, image_url) 
    for row_id, image_url in zip(df_images['RowId'], df_images['Top Review Image Url'])
]

# Unpack results into two new columns
df_images['img_resnet50_score'], df_images['img_resnet50_label'] = zip(*results)


0 https://www.airlinequality.com/wp-content/uploads/2019/11/20191102_123305-500x500.jpg
[895 908 404 403 657]
[5.190325  3.7582083 3.6045265 3.3716395 2.8360355]
['military aircraft' 'wing' 'airliner' 'aircraft carrier' 'missile']
13 https://www.airlinequality.com/wp-content/uploads/2018/06/IMG_1571-1-500x500.jpg
[636 748 785 464 885]
[6.6289973 4.485667  3.5766733 3.1832204 3.0856068]
['mail bag' 'purse' 'seat belt' 'buckle' 'velvet']
108 https://www.airlinequality.com/wp-content/uploads/2024/10/IMG_5873-500x500.jpeg
[609 545 791 790 581]
[5.0405984 4.7803736 4.7432065 4.4619155 4.0520296]
['jeep' 'electric fan' 'shopping cart' 'shopping basket' 'grille']
109 https://www.airlinequality.com/wp-content/uploads/2024/10/20241008_FRA-ATH-A3831_espresso-500x500.jpg
[967 968 504 910 785]
[7.355824  3.2939284 2.6654654 2.450601  2.326919 ]
['espresso' 'cup' 'coffee mug' 'wooden spoon' 'seat belt']
110 https://www.airlinequality.com/wp-content/uploads/2024/10/PXL_20241004_073123468-500x500.jpg

#### Persist to CSV file

In [248]:
df_images[['RowId','Airline Name','Review_Title','Review Date','Top Review Image Url',\
           'img_resnet50_score','img_resnet50_label']]\
    .to_csv("../data/silver_20250322_Airline_Reviews_Resnet50.csv")